In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

Stats_df = pd.read_csv(r"C:\Users\yazda\OneDrive\Desktop\documents\NBA_Stats.csv")

Stats_df = Stats_df.drop_duplicates()
Stats_df = Stats_df.fillna(0)
Stats_df = Stats_df.drop('Rk', axis = 1)
Stats_df = Stats_df.drop('Awards', axis = 1)
Stats_df = Stats_df.drop('Player-additional', axis = 1)

Stats_df['FG%'] = Stats_df['FG%'] * 100
Stats_df['FT%'] = Stats_df['FT%'] * 100
Stats_df['3P%'] = Stats_df['3P%'] * 100

Stats_df

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Shai Gilgeous-Alexander,26.0,OKC,PG,76.0,76.0,34.2,11.3,21.8,51.9,...,89.8,0.9,4.1,5.0,6.4,1.7,1.0,2.4,2.2,32.7
1,Giannis Antetokounmpo,30.0,MIL,PF,66.0,66.0,34.1,11.8,19.7,60.3,...,61.2,2.2,9.7,11.9,6.3,0.9,1.2,3.1,2.3,30.4
2,Nikola Jokić,29.0,DEN,C,68.0,68.0,36.8,11.3,19.6,57.5,...,80.8,2.9,9.9,12.8,10.2,1.8,0.6,3.3,2.3,29.8
3,Anthony Edwards,23.0,MIN,SG,77.0,77.0,36.4,9.1,20.4,44.7,...,83.6,0.8,4.9,5.7,4.5,1.1,0.6,3.2,1.9,27.6
4,Jayson Tatum,26.0,BOS,PF,71.0,71.0,36.5,9.2,20.4,45.3,...,81.3,0.7,8.0,8.7,6.0,1.1,0.5,2.9,2.2,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,Kris Murray,24.0,POR,SF,67.0,4.0,14.5,1.7,3.8,43.4,...,43.1,1.0,1.4,2.5,0.9,0.5,0.2,0.6,0.9,4.0
229,Nicolas Batum,36.0,LAC,PF,78.0,8.0,17.5,1.3,3.1,43.7,...,81.0,0.6,2.2,2.8,1.1,0.7,0.5,0.4,1.4,4.0
230,Steven Adams,31.0,HOU,C,57.0,3.0,13.6,1.6,2.9,54.6,...,46.2,2.9,2.8,5.6,1.1,0.4,0.5,0.9,1.0,3.9
231,Andre Jackson Jr.,23.0,MIL,SG,66.0,43.0,14.3,1.4,2.9,47.1,...,50.0,1.0,1.7,2.7,1.2,0.5,0.2,0.8,1.7,3.3


In [6]:
Team_Stats_df = pd.read_csv(r"C:\Users\yazda\OneDrive\Desktop\documents\TeamStats_NBA.csv")

Team_Stats_df = Team_Stats_df.drop_duplicates()
Team_Stats_df = Team_Stats_df.fillna(0)

team_columns_to_drop = set(['Rk','Unnamed: 17','Unnamed: 22', 'Unnamed: 27']) & set(Team_Stats_df.columns)
Team_Stats_df = Team_Stats_df.drop(columns = team_columns_to_drop)

Team_Stats_df

,Team,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,...,TOV%,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Arena,Attend.,Attend./G
0,OKC,24.7,67.0,14.0,67,14,12.84,-0.07,12.77,120.4,...,10.4,24.1,0.179,0.515,14.9,74.6,0.210,Paycom Center,754832,17973
1,CLE,26.6,64.0,17.0,62,19,9.75,-0.77,8.98,122.1,...,11.6,25.9,0.188,0.529,12.7,74.7,0.181,Rocket Arena,777280,19432
2,BOS,28.9,60.0,21.0,61,20,9.14,-0.72,8.41,120.7,...,10.7,25.6,0.170,0.523,11.6,76.2,0.154,TD Garden,766240,19156
3,MIN,27.2,48.0,33.0,53,28,4.93,0.25,5.18,116.6,...,13.0,25.7,0.198,0.532,13.2,75.0,0.179,Target Center,753271,18832
4,HOU,24.9,52.0,29.0,52,29,4.75,0.41,5.16,115.3,...,11.9,31.7,0.180,0.528,12.8,76.2,0.185,Toyota Center,698798,17470
5,LAC,29.7,49.0,32.0,52,29,4.65,0.15,4.80,114.9,...,13.4,24.4,0.201,0.535,13.6,77.4,0.189,Intuit Dome,679593,16575
6,MEM,24.7,47.0,34.0,51,30,4.48,-0.03,4.45,117.6,...,13.2,28.5,0.197,0.534,12.8,74.9,0.206,FedEx Forum,665273,16632
7,NYK,27.5,50.0,31.0,50,31,4.05,-0.43,3.62,118.5,...,11.9,26.0,0.185,0.548,13.0,74.4,0.175,Madison Square Garden (IV),811794,19800
8,DEN,27.0,49.0,32.0,49,32,3.75,0.02,3.78,119.8,...,12.5,26.7,0.199,0.542,11.3,74.7,0.174,Ball Arena,811211,19786
9,GSW,28.6,48.0,33.0,49,32,3.41,0.20,3.61,115.0,...,12.3,27.5,0.186,0.540,14.0,75.6,0.189,Chase Center,722560,18064


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Read and cleans the data
class DataLoader:
    def __init__(self, filepath):
        self.df = pd.read_csv(filepath)
        self.Team_df = pd.read_csv(TeamStats_path)
        self._clean_data()

    def _clean_data(self):
        self.df.drop_duplicates(inplace=True)
        self.df.fillna(0,inplace=True)
        columns_to_drop = set(['Rk', 'Awards', 'Player-additional']) & set(self.df.columns)
        self.df.drop(columns = columns_to_drop, inplace=True)

        self.Team_df.drop_duplicates(inplace=True)
        self.Team_df.fillna(0,inplace=True)
        columns_to_drop = set(['Rk','Unnamed: 17', 'Unnamed: 22', 'Unnamed: 27']) & set(self.df.columns)
        self.df.drop(columns = columns_to_drop, inplace=True)

    #gets the top n scorers
    def get_top_scorers(self, n=10):
        self.df['PTS'] = pd.to_numeric(self.df['PTS'], errors='coerce')
        top_players = self.df.sort_values(by='PTS', ascending=False).head(n)
        return top_players.reset_index(drop=True)


# Player Object
class Player:
    def __init__(self, data):           #Data parameter represents a row of the dataframe
        self.name = data['Player']
        self.pts = float(data['PTS'])
        self.fg = float(data['FG'])
        self.fga = float(data['FGA'])
        self.orb = float(data['ORB'])
        self.team_orb_rate = team_orb_rate

    @property
    def missed_shots(self):
        return max(self.fga - self.fg, 0)

    @property
    def kobe_assists(self):
        return self.missed_shots * self.team_orb_rate * self.team_pts_per_possession



# Kobe Assist calculator
class KobeAssistCalculator:
    def __init__(self, df):
        self.players = [
            Player(row,team_orb_rate, team_pts_per_possession ) 
            for _, row in df.iterrows()
            ]    #Creates a list of player objects

    def calculate_all(self):
        return {
            self.players[i].name: self.players[i].kobe_assists      #Creates a dictionary of player names and their Kobe assists
            for i in range(len(self.players))
        }


# Plotting results
class KobeAssistPlotter:
    @staticmethod
    def plot(kobe_dict):
        players = list(kobe_dict.keys())
        Kobe_assists = list(kobe_dict.values())
        plt.figure(figsize=(12, 6))
        plt.bar(players, Kobe_assists)
        plt.title("Estimated 'Kobe Assists'", fontsize=14)
        plt.ylabel("Kobe Assists (Points)")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.show()


In [ ]:
# Running everything
def main():
    file_path = r"C:\Users\yazda\OneDrive\Desktop\documents\NBA_Stats.csv"
    TeamStats_path = r"C:\Users\yazda\OneDrive\Desktop\documents\TeamStats_NBA.csv"
    loader = DataLoader(file_path)             
    top_scorers_df = loader.get_top_scorers()   

    # Example values for team ORB rate and points per possession
    team_orb_rate = 0.3  # Replace with actual value from Team_Stats_df
    team_pts_per_possession = 1.1  # Replace with actual value from Team_Stats_df

    calculator = KobeAssistCalculator(top_scorers_df)
    kobe_assists = calculator.calculate_all()

    while True:
        try:
            select = int(input("Press:\n1 - About\n2 - Top Scorers Kobe Assist\n3 - Display Bar Graph\n4 - Exit\n"))

            if select == 1:
                print("In 2012 Kirk GoldBerry Calculated the points generated by a team off of a single players missed shots\nHe tallied up the amount of times a single players misses led to scoring possessions by his team and \ncoined this term the 'Kobe assist' named after the legendary Kobe Bryant. Here in this project we have\ncalculated the number of kobe assists averaged per game by the top scorers in the NBA in the 2025 NBA season")

            elif select == 2:
                print("Estimated Kobe Assists:\n")
                for name, val in kobe_assists.items():
                    print(f"{name}: {val:.2f} points")
                    
                print("\n")

            elif select == 3:
                KobeAssistPlotter.plot(kobe_assists)

            elif select == 4:
                print("Exiting program.")
                break

            else:
                print("Please enter a number from 1 to 4.")
                
        except ValueError:
            print("Invalid input. Please enter a valid number.")

In [ ]:
if __name__ == "__main__":
    main()